In [1]:
import os
import sys
import random
sys.path.append('../..')

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K

import cr_interface as cri
import keras_utils as ku

Using TensorFlow backend.


In [2]:
SEED = 37
def reset_random():
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(SEED)
    random.seed(SEED)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    tf.set_random_seed(SEED)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

In [3]:
ku.applications

{'mobilenet': <keras_utils.Application at 0x127993748>,
 'mobilenetv2': <keras_utils.Application at 0x128396cf8>,
 'inceptionresnetv2': <keras_utils.Application at 0x128396c88>,
 'inceptionv3': <keras_utils.Application at 0x128396cc0>,
 'nasnet': <keras_utils.Application at 0x128396d30>,
 'resnet50': <keras_utils.Application at 0x128396d68>,
 'vgg16': <keras_utils.Application at 0x128396da0>,
 'vgg19': <keras_utils.Application at 0x128396dd8>,
 'xception': <keras_utils.Application at 0x128396e10>}

In [4]:
application = ku.applications['mobilenet']

In [5]:
splits = cri.DATA_DIRS.keys()

In [6]:
def get_generators():
    transform_parameters = {
        'zx': 0.6,
        'zy': 0.6,
    }
    zoom_gen = ImageDataGenerator()
    zoom = lambda x: zoom_gen.apply_transform(x, transform_parameters)

    generators = dict()
    for split in splits:
        if split == 'test':
            augment_kwargs = dict()
        else:
            augment_kwargs = dict(
                rotation_range=45,
                fill_mode='nearest'
            )

        generators[split] = ImageDataGenerator(
            **augment_kwargs,
            preprocessing_function=zoom)

    return generators

In [7]:
def get_iterators():
    generators = get_generators()

    iterators = dict()

    kwargs = dict(
        target_size=application.image_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        seed=SEED)

    for split, gen in generators.items():
        iterators[split] = gen.flow_from_directory(
            directory=cri.DATA_DIRS[split],
            **kwargs)
        
    return iterators

# Controlling Randomness
- New sequential model from seed reproducable
- Save/load/attach top model (with same, reproducable results)
- Save/load/attach randomly trained top model (testing)

## Bottleneck

In [8]:
def get_labels(iterator, multiplier=1):
    # reset seed parameters
    # note that you need to use the same iterator to reproduce order
    iterator.total_batches_seen = 0
    iterator.batch_index = 0
    
    labels = None
    for i, batch in enumerate(iterator):
        if i == len(iterator) * multiplier:
            break
        if labels is None:
            labels = np.array(batch[1])
        else:
            labels = np.append(labels, np.array(batch[1]), axis=0)
            
    return labels

In [9]:
LOADED = False

In [41]:
# create bottlenecks & save
iterators = get_iterators()
bottlenecks = dict()
labels = dict()
application.free_model()

kwargs = dict(
    verbose=1,
    workers=8,
    use_multiprocessing=True)

for split, it in iterators.items():
    bottlenecks[split] = application.get_model().predict_generator(it, steps=len(it), **kwargs)
    labels[split] = get_labels(it)
    
LOADED = True

Found 251 images belonging to 3 classes.
Found 472 images belonging to 3 classes.
Found 1682 images belonging to 3 classes.
loading mobilenet model
53/53 [==============================] - 199s 4s/step


In [42]:
# Save Bottlenecks
for split, data in bottlenecks.items():
    np.save(open('b_{}'.format(split), 'wb'), data)
    
for split, data in labels.items():
    np.save(open('l_{}'.format(split), 'wb'), data)

In [10]:
# load bottlenecks
bottlenecks = dict()
labels = dict()
for split in splits:
    bottlenecks[split] = np.load(open('b_{}'.format(split), 'rb'))
    labels[split] = np.load(open('l_{}'.format(split), 'rb'))

In [11]:
bottlenecks.keys()

dict_keys(['test', 'validation', 'train'])

In [12]:
labels.keys()

dict_keys(['test', 'validation', 'train'])

#### No problem with bottleneck save / load

In [14]:
def compile_model(model, lr=1.0e-4):
    sgd = optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=sgd,
        metrics=['accuracy'])

In [15]:
def load_model(compiled=True):
    model = Sequential()
    model.add(Flatten(input_shape=application.get_model().output_shape[1:]))
    model.add(Dense(1024,
                        activation='relu',
                        kernel_initializer=keras.initializers.glorot_uniform(seed=SEED)))
    model.add(Dropout(0.5,
                         seed=SEED))
    model.add(Dense(3, 
                        activation='softmax',
                        kernel_initializer=keras.initializers.glorot_uniform(seed=SEED)))

    if compiled:
        compile_model(model)
    
    return model

In [356]:
reset_random()
top_model = load_model()
bottle_predictions = top_model.predict(bottlenecks['test'], verbose=1)

251/251 [==============================] - 19s 75ms/step


In [52]:
reset_random()
top_model = load_model()
top_model.fit(bottlenecks['train'], labels['train'],
              validation_data=(bottlenecks['validation'], labels['validation']),
              shuffle=True,
              batch_size=32,
              epochs=10)
top_model.save_weights('temp.hdf5')

Train on 1682 samples, validate on 472 samples
Epoch 1/10
1682/1682 [==============================] - 26s 15ms/step - loss: 0.8244 - acc: 0.7331 - val_loss: 0.9222 - val_acc: 0.6992
Epoch 2/10
1682/1682 [==============================] - 21s 13ms/step - loss: 0.3152 - acc: 0.8787 - val_loss: 0.5314 - val_acc: 0.8051
Epoch 3/10
1682/1682 [==============================] - 22s 13ms/step - loss: 0.1893 - acc: 0.9275 - val_loss: 0.5551 - val_acc: 0.7924
Epoch 4/10
1682/1682 [==============================] - 21s 13ms/step - loss: 0.1318 - acc: 0.9536 - val_loss: 0.5208 - val_acc: 0.8008
Epoch 5/10
1682/1682 [==============================] - 21s 13ms/step - loss: 0.0997 - acc: 0.9649 - val_loss: 0.5012 - val_acc: 0.8178
Epoch 6/10
1682/1682 [==============================] - 21s 13ms/step - loss: 0.0749 - acc: 0.9768 - val_loss: 0.5622 - val_acc: 0.7860
Epoch 7/10
1682/1682 [==============================] - 21s 13ms/step - loss: 0.0670 - acc: 0.9768 - val_loss: 0.4945 - val_acc: 0.8220
E

In [36]:
reset_random()
top_model = load_model()
top_model.fit(bottlenecks['train'], labels['train'],
              validation_data=(bottlenecks['validation'], labels['validation']),
              shuffle=True,
              batch_size=32,
              epochs=1)
top_model.save_weights('temp.hdf5')

KeyError: 'train'

In [53]:
p1 = top_model.predict(bottlenecks['test'])

In [54]:
p1

array([[7.87683427e-01, 2.11111024e-01, 1.20558089e-03],
       [9.97439623e-01, 1.99679489e-04, 2.36069388e-03],
       [9.99309540e-01, 3.32358868e-05, 6.57224853e-04],
       [9.29683626e-01, 1.17713251e-04, 7.01986626e-02],
       [9.80435491e-01, 1.56818125e-02, 3.88259650e-03],
       [7.23895757e-03, 9.92738426e-01, 2.26626180e-05],
       [3.97614896e-01, 4.73181248e-01, 1.29203886e-01],
       [8.99769664e-02, 9.10013080e-01, 9.99573331e-06],
       [9.99305367e-01, 2.99559353e-04, 3.95048643e-04],
       [5.62784553e-01, 1.93225045e-03, 4.35283184e-01],
       [1.65104605e-02, 9.83261287e-01, 2.28235818e-04],
       [9.99631763e-01, 5.43060014e-06, 3.62770981e-04],
       [8.57918188e-02, 5.31356258e-04, 9.13676858e-01],
       [9.90000308e-01, 1.36223468e-06, 9.99830011e-03],
       [9.97777164e-01, 9.03564251e-06, 2.21376540e-03],
       [7.96337798e-02, 5.40413661e-04, 9.19825792e-01],
       [3.70797157e-01, 8.66230130e-02, 5.42579830e-01],
       [9.93926346e-01, 3.61958

In [16]:
top_model = load_model(compiled=False)
top_model.load_weights('temp.hdf5')
compile_model(top_model)

loading mobilenet model


In [17]:
len(bottlenecks['test'])

251

In [18]:
p2 = top_model.predict(bottlenecks['test'])
p2

In [60]:
p2

array([[7.87683427e-01, 2.11111024e-01, 1.20558089e-03],
       [9.97439623e-01, 1.99679489e-04, 2.36069388e-03],
       [9.99309540e-01, 3.32358868e-05, 6.57224853e-04],
       [9.29683626e-01, 1.17713251e-04, 7.01986626e-02],
       [9.80435491e-01, 1.56818125e-02, 3.88259650e-03],
       [7.23895757e-03, 9.92738426e-01, 2.26626180e-05],
       [3.97614896e-01, 4.73181248e-01, 1.29203886e-01],
       [8.99769664e-02, 9.10013080e-01, 9.99573331e-06],
       [9.99305367e-01, 2.99559353e-04, 3.95048643e-04],
       [5.62784553e-01, 1.93225045e-03, 4.35283184e-01],
       [1.65104605e-02, 9.83261287e-01, 2.28235818e-04],
       [9.99631763e-01, 5.43060014e-06, 3.62770981e-04],
       [8.57918188e-02, 5.31356258e-04, 9.13676858e-01],
       [9.90000308e-01, 1.36223468e-06, 9.99830011e-03],
       [9.97777164e-01, 9.03564251e-06, 2.21376540e-03],
       [7.96337798e-02, 5.40413661e-04, 9.19825792e-01],
       [3.70797157e-01, 8.66230130e-02, 5.42579830e-01],
       [9.93926346e-01, 3.61958

In [19]:
#iterators = get_iterators() reuse
reset_random()
model = Sequential()
application.free_model()
pre_model = application.get_model()
for layer in pre_model.layers:
    layer.trainable = False
model.add(pre_model)
top_model = load_model(compiled=False)
top_model.load_weights('temp.hdf5')
model.add(top_model)
compile_model(model)



loading mobilenet model


In [26]:
import time 
iterators = get_iterators()
stime = time.time()
p3 = model.predict_generator(iterators['test'], verbose=1)
print('elapsed time per image = %4.2f s' % ((time.time()-stime) / 251))
p3

Found 251 images belonging to 3 classes.
Found 472 images belonging to 3 classes.
Found 1682 images belonging to 3 classes.
8/8 [==============================] - 57s 7s/step
elapsed time per image = 0.23 s


array([[7.87683427e-01, 2.11111024e-01, 1.20558089e-03],
       [9.97439623e-01, 1.99679489e-04, 2.36069388e-03],
       [9.99309540e-01, 3.32358868e-05, 6.57224853e-04],
       [9.29683626e-01, 1.17713251e-04, 7.01986626e-02],
       [9.80435491e-01, 1.56818125e-02, 3.88259650e-03],
       [7.23895757e-03, 9.92738426e-01, 2.26626180e-05],
       [3.97614896e-01, 4.73181248e-01, 1.29203886e-01],
       [8.99769664e-02, 9.10013080e-01, 9.99573331e-06],
       [9.99305367e-01, 2.99559353e-04, 3.95048643e-04],
       [5.62784553e-01, 1.93225045e-03, 4.35283184e-01],
       [1.65104605e-02, 9.83261287e-01, 2.28235818e-04],
       [9.99631763e-01, 5.43060014e-06, 3.62770981e-04],
       [8.57918188e-02, 5.31356258e-04, 9.13676858e-01],
       [9.90000308e-01, 1.36223468e-06, 9.99830011e-03],
       [9.97777164e-01, 9.03564251e-06, 2.21376540e-03],
       [7.96337798e-02, 5.40413661e-04, 9.19825792e-01],
       [3.70797157e-01, 8.66230130e-02, 5.42579830e-01],
       [9.93926346e-01, 3.61958

## Second Tune Experiment

In [66]:
application.get_model().layers[-10:]

In [70]:
#iterators = get_iterators() reuse
reset_random()
model = Sequential()
application.free_model()
pre_model = application.get_model()
for layer in pre_model.layers[:-3]:
    layer.trainable = False
for layer in pre_model.layers[-3:]:
    layer.trainable = True 
model.add(pre_model)
top_model = load_model(compiled=False)
top_model.load_weights('temp.hdf5')
model.add(top_model)
compile_model(model, lr=1.0e-6)

iterators = get_iterators()
model.fit_generator(iterators['train'],
                           validation_data=iterators['validation'],
                           shuffle=True,
                           epochs=10)

#p3 = model.predict_generator(iterators['test'], verbose=1)
#p3

loading mobilenet model
Found 251 images belonging to 3 classes.
Found 472 images belonging to 3 classes.
Found 1682 images belonging to 3 classes.
Epoch 1/10
53/53 [==============================] - 353s 7s/step - loss: 2.2120 - acc: 0.4857 - val_loss: 0.4160 - val_acc: 0.8411
Epoch 2/10
53/53 [==============================] - 337s 6s/step - loss: 1.7622 - acc: 0.5220 - val_loss: 0.4681 - val_acc: 0.8347
Epoch 3/10
53/53 [==============================] - 351s 7s/step - loss: 1.6714 - acc: 0.5176 - val_loss: 0.4951 - val_acc: 0.7797
Epoch 4/10
53/53 [==============================] - 350s 7s/step - loss: 1.5028 - acc: 0.5556 - val_loss: 0.4505 - val_acc: 0.8263
Epoch 5/10
53/53 [==============================] - 347s 7s/step - loss: 1.4066 - acc: 0.5612 - val_loss: 0.4386 - val_acc: 0.8411
Epoch 6/10
53/53 [==============================] - 310s 6s/step - loss: 1.3445 - acc: 0.5888 - val_loss: 0.4858 - val_acc: 0.8178
Epoch 7/10
53/53 [==============================] - 212s 4s/step -

## Raw Image

In [327]:
#iterators = get_iterators() reuse
model = Sequential()
pre_model = application.get_model()
for layer in pre_model.layers:
    layer.trainable = False
model.add(pre_model)
top_model = load_model()
model.add(top_model)

sgd = optimizers.SGD(lr=1.0e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

image_predictions = model.predict_generator(iterators['train'], verbose=1)
    
if False:
    model.fit_generator(iterators['train'],
                               validation_data=iterators['validation'],
                               shuffle=True,
                               epochs=1)

KeyboardInterrupt: 

In [318]:
image_predictions

array([[0.86311513, 0.05984244, 0.07704248],
       [0.3802605 , 0.6050819 , 0.01465751],
       [0.5054542 , 0.18332636, 0.31121948],
       ...,
       [0.04806492, 0.22226857, 0.7296665 ],
       [0.17853361, 0.1083675 , 0.7130988 ],
       [0.17712007, 0.07037111, 0.75250876]], dtype=float32)

In [319]:
#iterators = get_iterators() reuse
model = Sequential()
pre_model = application.get_model()
for layer in pre_model.layers:
    layer.trainable = False
model.add(pre_model)
top_model = load_model()
model.add(top_model)
model.summary()

sgd = optimizers.SGD(lr=1.0e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

image_predictions = model.predict_generator(iterators['train'], verbose=1)
    
if False:
    model.fit_generator(iterators['train'],
                               validation_data=iterators['validation'],
                               shuffle=True,
                               epochs=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
sequential_83 (Sequential)   (None, 3)                 51384323  
Total params: 54,613,187
Trainable params: 51,384,323
Non-trainable params: 3,228,864
_________________________________________________________________
53/53 [==============================] - 200s 4s/step


In [322]:
model = Sequential()
pre_model = application.get_model()
for layer in pre_model.layers:
    layer.trainable = False
model.add(pre_model)
sgd = optimizers.SGD(lr=1.0e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

image_predictions = model.predict_generator(iterators['test'], verbose=1)
image_predictions

8/8 [==============================] - 34s 4s/step


array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          9.6332407e-01, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          2.6831522e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, .

In [323]:
model = Sequential()
pre_model = application.get_model()
for layer in pre_model.layers:
    layer.trainable = False
model.add(pre_model)
sgd = optimizers.SGD(lr=1.0e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

image_predictions0 = model.predict_generator(iterators['test'], verbose=1)
image_predictions0

8/8 [==============================] - 34s 4s/step


array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          9.6332407e-01, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          2.6831522e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, .

In [324]:
model = Sequential()
pre_model = application.get_model()
model.add(pre_model)
sgd = optimizers.SGD(lr=1.0e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

image_predictions1 = model.predict_generator(iterators['test'], verbose=1)
image_predictions1

8/8 [==============================] - 35s 4s/step


array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          9.6332407e-01, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          2.6831522e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, .

In [320]:
image_predictions

array([[0.2940929 , 0.2536063 , 0.45230082],
       [0.27705055, 0.6346916 , 0.0882578 ],
       [0.20264404, 0.05492963, 0.7424263 ],
       ...,
       [0.2867003 , 0.03035819, 0.68294156],
       [0.10176496, 0.06876495, 0.82947004],
       [0.09121902, 0.02252517, 0.8862558 ]], dtype=float32)

In [279]:
#iterators = get_iterators() reuse
for it in iterators:
    it.batch_index = 0
    it.total_batches_seen = 0
model = Sequential()
pre_model = application.get_model()
for layer in pre_model.layers:
    layer.trainable = False
model.add(pre_model)
top_model = load_model(compiled=False)
model.add(top_model)
model.summary()
for layer in model.layers:
    print(layer)
    print(layer.trainable)

if False:
    sgd = optimizers.SGD(lr=1.0e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=sgd,
        metrics=['accuracy'])

    model.fit_generator(iterators['train'],
                               validation_data=iterators['validation'],
                               shuffle=True,
                               epochs=1)

AttributeError: 'str' object has no attribute 'batch_index'

In [267]:
top_model.evaluate(bottlenecks['test'], labels['test'])

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.